In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <stdio.h>
#include <stdlib.h>

struct timespec getDiffTime(struct timespec *startTime, struct timespec *endTime)
{
	struct timespec diff;                                                                         
	diff.tv_sec = endTime->tv_sec - startTime->tv_sec;                                            
	diff.tv_nsec = endTime->tv_nsec - startTime->tv_nsec;                                         
	if (diff.tv_nsec < 0) {                                                                       
		diff.tv_nsec += 1.0e9;                                                                
		diff.tv_sec--;                                                                        
	}                                                                                             
	return diff;                                                                                  
} 

// Uncomment the following statement to show the debug information
//#define DEBUG 1
#ifdef DEBUG
#define DEBUG_PRINT(fmt, args...) printf(fmt, ## args)
#else
#define DEBUG_PRINT(...) 
#endif

#define BLOCK_SIZE 32
#define COARSE_FACTOR 1

/*
 * Figure 5.13: A simple sum reduction kernel.
 * The sum reduction kernel function running in a single block without requirements on control divergences
 * The number of threads is the number n of elements in the vector.
 */
__global__ void sumRedKernel0(float* d_vec, unsigned int n)
{
	extern __shared__ float ds_partialSum[];
	unsigned int t = threadIdx.x;
	ds_partialSum[t] = d_vec[t];
	DEBUG_PRINT("ds_partialSum[%d] = %.1f\n", t, ds_partialSum[t]);
	for (unsigned int stride = 1; stride < n; stride *= 2) {
		__syncthreads();
		if (t % (2 * stride) == 0 && t + stride < n) {
			ds_partialSum[t] += ds_partialSum[t + stride];
			DEBUG_PRINT("stride = %d: ds_partialSum[%d] = %.1f\n", 
					stride, t, ds_partialSum[t]);
			DEBUG_PRINT("stride = %d: ds_partialSum[%d] = %.1f\n", 
					stride, t + stride, ds_partialSum[t + stride]);
		}
	}
	if (t == 0)
		d_vec[0] = ds_partialSum[0];
}

/* 
 * Figure 5.15: A kernel with fewer thread divergence.
 * The sum reduction kernel function running in a single block with fewer control divergences
 * The number of threads is the minimum power of 2 not smaller than the number n of elements in the vector.
 * Increase the vector size to be 2 raised to the power of an positive integer by padding with 0s
 */
__global__ void sumRedKernel1(float* d_vec, unsigned int n)
{
	extern __shared__ float ds_partialSum[];
	unsigned int t = threadIdx.x;
	if (t < n)
		ds_partialSum[t] = d_vec[t];
	else
		ds_partialSum[t] = 0.0;    // padding with 0s
	DEBUG_PRINT("ds_partialSum[%d] = %.1f\n", t, ds_partialSum[t]);
	for (unsigned int stride = blockDim.x / 2; stride >= 1; stride /= 2) {
		__syncthreads();
		if (t < stride) {
			ds_partialSum[t] += ds_partialSum[t + stride];
			DEBUG_PRINT("stride = %d: ds_partialSum[%d] = %.1f\n", 
					stride, t, ds_partialSum[t]);
			DEBUG_PRINT("stride = %d: ds_partialSum[%d] = %.1f\n", 
					stride, t + stride, ds_partialSum[t + stride]);
		}
	}
	if (t == 0)
		d_vec[0] = ds_partialSum[0];
}

/* 
 * Exercise 5.1: Modify the kernel in Figure 5.13 to eliminate the waste of half of the threads in each block.
 * The sum reduction kernel function running in a single block without requirements on control divergences
 */
__global__ void sumRedKernel2(float* d_vec, unsigned int n)
{
	extern __shared__ float ds_partialSum[];
	unsigned int t = threadIdx.x;
	ds_partialSum[t] = d_vec[2 * t];
	DEBUG_PRINT("ds_partialSum[%d] = %.1f\n", t, ds_partialSum[t]);
	if (2 * t + 1 < n)
		ds_partialSum[t] += d_vec[2 * t + 1];
	DEBUG_PRINT("ds_partialSum[%d] = %.1f\n", t, ds_partialSum[t]);
	for (unsigned int stride = 1; stride < blockDim.x; stride *= 2) {
		__syncthreads();
		if (t % stride == 0 && 2 * t + stride < blockDim.x) {
			ds_partialSum[2 * t] += ds_partialSum[2 * t + stride];
			DEBUG_PRINT("stride = %d: ds_partialSum[%d] = %.1f\n", 
					stride, 2 * t, ds_partialSum[2 * t]);
			DEBUG_PRINT("stride = %d: ds_partialSum[%d] = %.1f\n", 
					stride, 2 * t + stride, ds_partialSum[2 * t + stride]);
		}
	}
	if (t == 0)
		d_vec[0] = ds_partialSum[0];
}

/*
 * Exercise 5.1: Modify the kernel in Figure 5.15 to eliminate the waste of half of the threads in each block.
 * The sum reduction kernel function running in a single block with fewer control divergences
 * Increase the vector size to be 2 raised to the power of an positive integer by padding with 0s
 */
__global__ void sumRedKernel3(float* d_vec, unsigned int n)
{
	extern __shared__ float ds_partialSum[];
	unsigned int t = threadIdx.x;
	ds_partialSum[t] = d_vec[t];
	DEBUG_PRINT("ds_partialSum[%d] = %.1f\n", t, ds_partialSum[t]);
	if (blockDim.x + t < n)
		ds_partialSum[t] += d_vec[blockDim.x + t];
	DEBUG_PRINT("ds_partialSum[%d] = %.1f\n", t, ds_partialSum[t]);
	for (unsigned int stride = blockDim.x / 2; stride >= 1; stride /= 2) {
		__syncthreads();
		if (t < stride) {
			ds_partialSum[t] += ds_partialSum[t + stride];
			DEBUG_PRINT("stride = %d: ds_partialSum[%d] = %.1f\n", 
					stride, t, ds_partialSum[t]);
			DEBUG_PRINT("stride = %d: ds_partialSum[%d] = %.1f\n", 
					stride, t + stride, ds_partialSum[t + stride]);
		}
	}
	if (t == 0)
		d_vec[0] = ds_partialSum[0];
}

/*
 * Exercise 5.3: 
 * The scalable sum reduction kernel function without requirements on control divergences
 * 1) Add the statements that load a section of the input array from global memory to shared memory.
 * 2) Use blockIdx.x to allow multiple blocks to work on different sections of the input array. 
 * 3) Write the reduction value for the section to a location according to the blockIdx.x, 
 *    so that all blocks will deposit their section reduction value to the lower part of the input array in global memory.
 */
__global__ void sumRedKernel4(float* d_vec, unsigned int n, unsigned int step)
{
	extern __shared__ float ds_partialSum[];
	unsigned int s = blockIdx.x * (2 * blockDim.x);
	unsigned int t = threadIdx.x;
	if (s + 2 * t < n) {
		ds_partialSum[t] = d_vec[step * (s + 2 * t)];
		DEBUG_PRINT("step = %d: d_vec[%d] = %.1f\n", step, step * (s + 2 * t), 
				d_vec[step * (s + 2 * t)]);
	}
	if (s + 2 * t + 1 < n) {
		ds_partialSum[t] += d_vec[step * (s + 2 * t + 1)];
		DEBUG_PRINT("step = %d: d_vec[%d] = %.1f\n", step, step * (s + 2 * t + 1), 
				d_vec[step * (s + 2 * t + 1)]);
	}
	unsigned int num;
	if (n - s >= 2 * blockDim.x)
		num = blockDim.x;
	else if ((n - s) % 2 == 0)
		num = (n - s) / 2;
	else
		num = (n - s) / 2 + 1;
	for (unsigned int stride = 1; stride < num; stride *= 2) {
		__syncthreads();
		if (t % stride == 0 && 2 * t + stride < num) {
			ds_partialSum[2 * t] += ds_partialSum[2 * t + stride];
			DEBUG_PRINT("step = %d, stride = %d: d_vec[%d] = %.1f\n", 
					step, stride, step * (s + 2 * t), ds_partialSum[2 * t]);
			DEBUG_PRINT("step = %d, stride = %d: d_vec[%d] = %.1f\n", 
					step, stride, step * (s + 2 * t + stride), 
					ds_partialSum[2 * t + stride]);
		}
	}
	if (t == 0)
		d_vec[step * s] = ds_partialSum[0];
}

/*
 * Exercise 5.3: 
 * The scalable sum reduction kernel function with fewer control divergences using padding with 0s
 * 1) Add the statements that load a section of the input array from global memory to shared memory.
 * 2) Use blockIdx.x to allow multiple blocks to work on different sections of the input array. 
 * 3) Write the reduction value for the section to a location according to the blockIdx.x, 
 *    so that all blocks will deposit their section reduction value to the lower part of the input array in global memory.
 */
__global__ void sumRedKernel5(float* d_vec, unsigned int n, unsigned int step)
{
	extern __shared__ float ds_partialSum[];
	unsigned int s = blockIdx.x * (2 * blockDim.x);
	unsigned int t = threadIdx.x;
	if (s + t < n)
		ds_partialSum[t] = d_vec[step * (s + t)];
	else
		ds_partialSum[t] = 0.0;
	if (s + blockDim.x + t < n)
		ds_partialSum[t] += d_vec[step * (s + blockDim.x + t)];
	DEBUG_PRINT("step = %d: d_vec[%d] = %.1f\n", step, step * (s + t), d_vec[step * (s + t)]);
	DEBUG_PRINT("step = %d: d_vec[%d] = %.1f\n", step, step * (s + blockDim.x + t), d_vec[step * (s + blockDim.x + t)]);
	for (unsigned int stride = blockDim.x / 2; stride >= 1; stride /= 2) {
		__syncthreads();
		if (t < stride) {
			ds_partialSum[t] += ds_partialSum[t + stride];
			DEBUG_PRINT("step = %d, stride = %d: d_vec[%d] = %.1f\n", 
					step, stride, step * (s + t), ds_partialSum[t]);
			DEBUG_PRINT("step = %d, stride = %d: d_vec[%d] = %.1f\n", 
					step, stride, step * (s + t + stride), ds_partialSum[t + stride]);
		}
	}
	if (t == 0)
		d_vec[step * s] = ds_partialSum[0];
}

/*
 * Figure 10.15: A segmented multi-block sum reduction kernel using atomic operations and thread coarsening.
 */
__global__ void sumRedKernel6(float* d_vec, float* d_sum, unsigned int n)
{
	extern __shared__ float ds_partialSum[];
	unsigned int s = blockIdx.x * (2 * blockDim.x) * COARSE_FACTOR;
	unsigned int t = threadIdx.x;
	ds_partialSum[t] = 0.0;
	for (int tileIdx = 0; tileIdx < 2 * COARSE_FACTOR && s + tileIdx * blockDim.x + t < n; tileIdx++) {
		ds_partialSum[t] += d_vec[s + tileIdx * blockDim.x + t];
	}
	DEBUG_PRINT("block = %d: ds_partialSum[%d] = %.1f\n", blockIdx.x, t, ds_partialSum[t]);
	for (unsigned int stride = blockDim.x / 2; stride >= 1; stride /= 2) {
		__syncthreads();
		if (t < stride) {
			ds_partialSum[t] += ds_partialSum[t + stride];
			DEBUG_PRINT("block = %d, stride = %d: ds_partialSum[%d] = %.1f\n", 
					blockIdx.x, stride, t, ds_partialSum[t]);
			DEBUG_PRINT("block = %d, stride = %d: ds_partialSum[%d] = %.1f\n", 
					blockIdx.x, stride, t + stride, ds_partialSum[t + stride]);
		}
	}
	if (t == 0) {
		DEBUG_PRINT("block = %d: d_sum = %.1f\n", blockIdx.x, *d_sum);
		atomicAdd(d_sum, ds_partialSum[0]);
		DEBUG_PRINT("block = %d: ds_partialSum[0] = %.1f, d_sum = %.1f\n", blockIdx.x, ds_partialSum[0], *d_sum);
	}
}

float sumRed(float* h_vec, unsigned int n, unsigned int version)
{
	unsigned int size = n * sizeof(float);
	float *d_vec, *d_sum;
	float h_sum = 0.0;

	cudaMalloc((void **) &d_vec, size);
	cudaMalloc((void **) &d_sum, sizeof(float));

	cudaMemcpy(d_vec, h_vec, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_sum, &h_sum, sizeof(float), cudaMemcpyHostToDevice);

	unsigned int blockSize = BLOCK_SIZE, gridSize, sharedMemSize, iter = 0, step = 1;
	switch(version) {
		case 0:
			gridSize = 1;
			blockSize = n;
			sharedMemSize = n * sizeof(float); 
			sumRedKernel0<<<gridSize, blockSize, sharedMemSize>>>(d_vec, n);
			break;
		case 1:
			gridSize = 1;
			blockSize = 1 << ((int) ceil(log2(n)));    // the minimum power of 2 not smaller than n
			sharedMemSize = blockSize * sizeof(float); 
			sumRedKernel1<<<gridSize, blockSize, sharedMemSize>>>(d_vec, n);
			break;
		case 2:
			gridSize = 1;
			blockSize = ceil(n / 2.0);
			sharedMemSize = blockSize * sizeof(float); 
			sumRedKernel2<<<gridSize, blockSize, sharedMemSize>>>(d_vec, n);
			break;
		case 3:
			gridSize = 1;
			blockSize = 1 << ((int) ceil(log2(n)) - 1);
			sharedMemSize = blockSize * sizeof(float); 
			sumRedKernel3<<<gridSize, blockSize, sharedMemSize>>>(d_vec, n);
			break;
		case 4:
			/*
			 * Exercise 5.4: 
			 * Use a loop to repeatedly invoke the kernel function sumRedKernel4
			 * with adjusted execution configuration parameter values so that
			 * the reduction result for the input array will eventually be produced.
			 */
			do {
				if (BLOCK_SIZE < ceil(n / 2.0)) {
					blockSize = BLOCK_SIZE;
					gridSize = ceil(n / (2.0 * blockSize));
				} else {
					gridSize = 1;
					blockSize = ceil(n / 2.0);
				}
				sharedMemSize = blockSize * sizeof(float);
				step = (unsigned int) pow(2 * BLOCK_SIZE, iter);
				sumRedKernel4<<<gridSize, blockSize, sharedMemSize>>>(d_vec, n, step);
				iter++;
				n = gridSize;
			} while (n > 1);
			break;
		case 5:
		default:
			/*
			 * Exercise 5.4: 
			 * Use a loop to repeatedly invoke the kernel function sumRedKernel5
			 * with adjusted execution configuration parameter values so that
			 * the reduction result for the input array will eventually be produced.
			 */
			do {
				if (BLOCK_SIZE < ceil(n / 2.0)) {
					blockSize = 1 << (int) ceil(log2(BLOCK_SIZE));
					gridSize = ceil(n / (2.0 * blockSize));
					step = (unsigned int) pow(2 * blockSize, iter);
				} else {
					step = (unsigned int) pow(2 * blockSize, iter);
					gridSize = 1;
					blockSize = 1 << ((int) ceil(log2(n)) - 1);
				}
				sharedMemSize = blockSize * sizeof(float); 
				sumRedKernel5<<<gridSize, blockSize, sharedMemSize>>>(d_vec, n, step);
				iter++;
				n = gridSize;
			} while (n > 1);
		case 6:
			blockSize = 1 << (int) ceil(log2(BLOCK_SIZE));
			gridSize = ceil(n / (2.0 * blockSize * COARSE_FACTOR));
			sharedMemSize = blockSize * sizeof(float); 
			sumRedKernel6<<<gridSize, blockSize, sharedMemSize>>>(d_vec, d_sum, n);
			break;
	}

	if (version == 6)
		cudaMemcpy(&h_sum, d_sum, sizeof(float), cudaMemcpyDeviceToHost);
	else
		cudaMemcpy(&h_sum, d_vec, sizeof(float), cudaMemcpyDeviceToHost);

	cudaFree(d_vec);
	cudaFree(d_sum);
	return h_sum;
}

int main(int argc, char *argv[])
{
	int version = 0;
	if (argc > 1) {
		version = atoi(argv[1]);
		printf("Kernel version = %d\n", version);
	}

	printf("Enter the number of elements to be summed up: ");
	unsigned int n;
	scanf("%d", &n);

	float *h_vec;
	h_vec = (float *) malloc(n * sizeof(float));

	for(int i = 0; i < n; i++)
		h_vec[i] = 2.0;

	struct timespec startTime;
	clock_gettime(CLOCK_REALTIME, &startTime);
	float sum = sumRed(h_vec, n, version);
	struct timespec endTime;
	clock_gettime(CLOCK_REALTIME, &endTime);

	printf("sum = %.1f\n", sum);
	struct timespec diffTime = getDiffTime(&startTime, &endTime);
	printf("Execution time: %ld s and %ld us.\n", diffTime.tv_sec, diffTime.tv_nsec / (long) 1000);

	free(h_vec);
	return 0;
}